### Loading model from 🤗

In [1]:
!nvidia-smi

Fri Jun 16 16:31:17 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.43.04    Driver Version: 515.43.04    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-DGXS...  On   | 00000000:07:00.0 Off |                    0 |
| N/A   31C    P0    36W / 300W |     28MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-DGXS...  On   | 00000000:08:00.0 Off |                    0 |
| N/A   

In [2]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(".")
from utils import *


In [2]:
from huggingface_hub import interpreter_login

interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Y

In [4]:
import torch
import textwrap
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0, 1, 2"
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda"

EVAL_PROMPTS = [
    "Your are an Pytorch expert. Question: How to create an attention layer in pytorch? \n\n Answer: ",
    "Question: I recieve TypeError with a following code: ```a=[1, 2, 3]    a.extend(4)```. How can I fix it?  \n\n Answer: ",
    "Question: What library would you recommend for visualizing 3D points in python? \n\n Answer: ",
]

def generate_eval(model: AutoModelForCausalLM, tokenizer: AutoTokenizer):
    print("Starting Evaluation...")
    model = model.to(device)
    for eval_prompt in EVAL_PROMPTS:
        batch = tokenizer(eval_prompt, return_tensors="pt").to(device)

        with torch.cuda.amp.autocast():
            output_tokens = model.generate(**batch, max_new_tokens=128, do_sample=True, temperature=0.5, top_p=0.75)

        print("\n\n", textwrap.fill(tokenizer.decode(output_tokens[0], skip_special_tokens=False)))
        print("*"*100)
        
model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B", torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B", add_eos_token=False, add_bos_token=False)

In [5]:
tokenizer.pad_token = tokenizer.eos_token
generate_eval(model, tokenizer)

Starting Evaluation...


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




 Your are an Pytorch expert. Question: How to create an attention layer
in pytorch?    Answer:   The attention layer is a function that takes
a sequence of inputs and outputs a sequence of outputs. The output of
the attention layer is a sequence of attention scores. The attention
score is a scalar value that is a weighted sum of the input sequence.
The attention score is computed by the following formula:
attention_score = attention_layer(input_sequence, attention_weights)
The attention layer takes a sequence of inputs and outputs a sequence
of outputs. The output of the attention layer is a sequence of
attention scores. The attention score is a scalar value that is a
weighted sum of the input sequence
****************************************************************************************************


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




 Question: I recieve TypeError with a following code: ```a=[1, 2, 3]
a.extend(4)```. How can I fix it?     Answer:   You can use the extend
method of a list.  ```python a = [1, 2, 3] a.extend(4)  print(a) # [1,
2, 3, 4] ```  ## 3.6.2  ```python a = [1, 2, 3] a.extend(4)  print(a)
# [1, 2, 3, 4] ```  ## 3.6.3  ```python a = [1, 2, 3
****************************************************************************************************


 Question: What library would you recommend for visualizing 3D points
in python?    Answer:   I would recommend using the pyOpenGL library.
A:  I would recommend using the pyOpenGL library.   A:  You can use
pyOpenGL to create a 3D model of a point. import pyOpenGL  point =
pyOpenGL.Point(100, 100)  # Draw the model
gl.glDrawArrays(gl.GL_POINTS, 0, 1)  # Set the model to the modelview
matrix gl.glMatrixMode(gl.GL_MODELVIEW) gl.glLoad
****************************************************************************************************


* **GPT-Neo was not trained to be ChatBot**
* **We can solve this by finetuning using PEFT & RLHF**